In [1]:
import os
import csv
import paramiko
import gzip

import pymysql
from sshtunnel import SSHTunnelForwarder

from tqdm import tqdm

from datetime import timedelta, date, datetime

import pandas as pd

from collections import defaultdict
import re
from shutil import copyfile



## Basics

In [4]:
#Inputs

database = "clinical_merge_v5_2020q4"
new_rels = True

#Build from ccsr
ped_data = '/data1/home/kel2158/Projects/pedigree_v3/data/final_results/'
output_folder= '/home/kel2158/Projects/COVIDHerit/data/12_29_21/'
hhid_folder = 'home/kel2158/Projects/hhid_update/data/'

#Input files
finename_fam_input = ped_data +"family_v3.txt.gz"
filename_hhid =  hhid_folder+'hhid_trimmed_notcln.txt.gz'

#Output filenames
filename_demo = output_folder+ 'demo.txt'
filename_covariates = output_folder+'covariates_building_race.txt'
filename_covariates_raceonly = output_folder+'covariates_race.txt'

filename_pheno_binary = output_folder+'covid_binary.txt'
filename_pheno_quant = output_folder+'covid_quant.txt'


fam_count_file = output_folder + "/fam_size.txt"

#Variables
#Patient must have died within 90 days of latest positve test
death_cutoff = datetime.timedelta(days=90)
intub_cutoff = datetime.timedelta(days=12)

In [5]:
def return_sign_in(login_loc = '/home/kel2158/login.txt'):
    login_dict = dict()
    with open(login_loc, "r") as file:
        for line in file.readlines():
            login_dict[ line.split(",")[0]] = line.split(",")[1].strip()
    return(login_dict)

In [6]:
login_dict = return_sign_in()

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect('10.144.220.21', username=login_dict['user'], password=login_dict['mimir_pw'])


In [7]:
# Make sure to call server.stop() when you want to disconnect
# after calling db.close()
server = SSHTunnelForwarder('10.144.220.21',
                            ssh_username='kel2158',
                            ssh_password=login_dict['mimir_pw'],
                            remote_bind_address=('127.0.0.1', 3306))
server.start()

db = pymysql.connect(host='127.0.0.1',
                     port=server.local_bind_port,
                     user=login_dict['user'],
                     password=login_dict["db_pw"],
                     db=database)

cur = db.cursor()

cur.execute( "SET sql_mode=(SELECT REPLACE(@@sql_mode,'ONLY_FULL_GROUP_BY',''));" )

0

## Pull lab and family data

In [8]:
#Read in family file 
file_name = finename_fam_input
ftp = ssh.open_sftp()
file = ftp.file(file_name, "r+")
gfile = gzip.open(file)

fam_df = pd.read_csv(gfile,sep='\t')

ftp.close()
ssh.close()
mrn2famid = dict(zip (fam_df["ptid"],fam_df["famid"]))

In [154]:
#Pull patients who have had positive covid tests

pull_data_query = """
DROP TABLE IF EXISTS user_kel2158.covid_lab_table;
"""
cur.execute(pull_data_query)


pull_data_query = """ CREATE TABLE user_kel2158.covid_lab_table AS
	SELECT  DISTINCT l.pat_mrn_id, m.order_time, l.order_proc_id,  l.result_date, l.ord_value, l.person_id_2020, l.person_id_2019,
	CASE
	WHEN l.ord_value IN ("Detected",
        "Detected A",
        "Detected..",
        "POSITIVE",
        "Postive",
        "Postivie A",
        "Yes"
        ) THEN 'pos'
	WHEN l.ord_value IN ("Not Detected",
        "Negative",
        "No",
        "Not Detected (External)",
        "SARS-CoV-2 neg",
        "neg",
        "Non detected",
        "Undetected",
        "Not Dectected",
        "NotDetectd",
        "Negatvie",
        "NOT_DETECT",
        "notdetected",
        "Not Deteceted",
        "Note Detected",
        "not dtected",
        "not  detected") THEN 'neg'
	END AS test_result
    FROM covid_labs l,
    covid_measurements m
    WHERE 
	(m.procedure_name like '%SARS%'
	OR m.component_name like '%SARS%'
    OR m.component_loinc_code = 94500)  AND m.order_proc_id = l.order_proc_id
;
"""

cur.execute(pull_data_query)

pull_data_query = "CREATE INDEX pat_mrn_id ON user_kel2158.covid_lab_table (pat_mrn_id);"
cur.execute(pull_data_query)

496876

In [9]:
#Get patients with an icd code for history of covid 
pull_data_query = """
SELECT DISTINCT p.pat_mrn_id
FROM clinical_merge_v5_2020q4.covid_patients p
WHERE icd10_code = 'Z86.16';

"""
cur.execute(pull_data_query)

rawdata = cur.fetchall()

hist_cov_ptid = set([e[0] for e in rawdata])

In [11]:
#Pull patients who have had positive covid tests

pull_data_query = """SELECT DISTINCT pat_mrn_id, order_time
FROM user_kel2158.covid_lab_table
WHERE test_result = 'pos';
"""

cur.execute(pull_data_query)
rawdata = cur.fetchall()

p_df = pd.DataFrame(list( rawdata ),columns=["ptid","order_time"])

cases = set([e[0] for e in rawdata])

p_df["value"] = 1

p_df["famid"] = p_df["ptid"].map(mrn2famid)

p_df = p_df[ p_df["famid"].notnull() ] 

In [12]:
#Pull patients who have had negative covid tests

pull_data_query = """SELECT DISTINCT pat_mrn_id
FROM user_kel2158.covid_lab_table
WHERE test_result = 'neg'
;
"""

cur.execute(pull_data_query)
rawdata = cur.fetchall()

intermediate_controls = set([e[0] for e in rawdata])

In [18]:
#Get familial degrees
pull_data_query = """
SELECT r.mrn, r.relation_mrn, d.relationship, d.degree
FROM clinical_relationships_v3.all_relationships_to_generate_pedigree_file r,
user_kel2158.covid_lab_table c1,
user_kel2158.covid_lab_table c2,
clinical_relationships.relationships_degree d
WHERE r.mrn = c1.pat_mrn_id AND
r.relation_mrn = c2.pat_mrn_id AND
r.relationship = d.relationship
;
"""

cur.execute(pull_data_query)
rawdata = cur.fetchall()

relationship_df = pd.DataFrame(rawdata,columns=["mrn","rel_mrn","relationship","degree"])



## Functions

In [22]:
def pull_fam_count( pheno, df, print_res = True, \
                   return_fams_df=False, type_pheno="binary_withproband",
                   return_patient_limited_df = False ):
    
    """Returns the number of families in a phenotype for SOLAR based on pheno 
     Needs phenotype column, famid column and ptid column"""    
    
    #subset on phenotype of interest and those in our pedigree
    df_phenosub = df[ (df["pheno"] == pheno ) 
                       &  (~df["famid"].isnull())] 
    
    #Get the total number of  members in each fam and then number of cases
    df_phenogrouped = df_phenosub.groupby( 'famid' )\
        .agg({ "ptid":"count","value":sum })\
        .reset_index()\
        .rename(columns={ "ptid":"num_fam_members","value":"num_cases" })
    
    #Subset to fams that will be included in SOLAR
    #ie has more than one family member an at least one case in each family
    if type_pheno == "binary_withproband":
        solar_fams_df = df_phenogrouped[ (df_phenogrouped["num_fam_members"] > 1) \
                                        & (df_phenogrouped["num_cases"] > 0) ]
    
    else:
         solar_fams_df = df_phenogrouped[ (df_phenogrouped["num_fam_members"] > 1)]
    
    solar_limited_df = df_phenosub[ df_phenosub["famid"].isin(solar_fams_df["famid"].unique())]
    solar_limited_df.drop_duplicates(inplace=True)
    solar_patients = set( solar_limited_df["ptid"] )
    
    
    
    if print_res:
            print(f'There are {len(solar_fams_df)} SOLAR families')
    
    if return_fams_df:
        return solar_fams_df, solar_patients
    
    elif return_patient_limited_df == True:
        return solar_limited_df
    
    else:
        return(len( solar_fams_df) )
    

In [24]:
def write_famcount(count, pheno):
    
    orig_file = fam_count_file
    temp_file = fam_count_file[:-4]+"_temp.txt"
    
    #Copy the original file into a temp file
    copyfile(orig_file, temp_file)  
        
    #Delete original file
    os.remove(orig_file)
    
    bad_words = [pheno]
    
    #exclude pheno if it's already there
    with open(temp_file) as oldfile, open(orig_file, 'w') as newfile:
        for line in oldfile:
            if not any(bad_word in line for bad_word in bad_words):
                newfile.write(line)
    
    #append pheno to end of updated file
    with open(orig_file, "a") as file_object:
        #append pheno to end of file
        file_object.write(pheno+" "+str(count))
        file_object.write("\n")
    
    file_object = open(orig_file, 'a')
    
    #Delete temporary file
    os.remove(temp_file)

## Pheno: Infection

In [25]:
#Controls Overall: had a test but never positive
controls = intermediate_controls-cases

#Remove those patients with history of covid icd code
#Note, don't use covid only icd codes because gold standard is pcr and need timing of infection
controls = controls - hist_cov_ptid
controls = set(controls)

pheno_df = pd.concat( [  pd.DataFrame(zip(cases,len(cases)*["covid"],len(cases)*[1])),
                       pd.DataFrame(zip(controls,len(controls)*['covid'],len(controls)*[0])) ])
pheno_df.rename(columns={0:"ptid",1:"pheno",2:"value"},inplace=True)

#Drop duplicates not necessary, select distinct negatives and select min and max for positives


pheno_df["famid"] = pheno_df["ptid"].map(mrn2famid)

In [26]:
len(controls)

181397

In [45]:
len(cases)

17875

In [28]:
pull_data_query = """ SELECT DISTINCT pat_mrn_id, death_date
FROM covid_persons
WHERE death_date IS NULL; """



cur.execute(pull_data_query)
rawdata = cur.fetchall()
deaths_df = pd.DataFrame(list(rawdata),columns=["mrn","death_date"])

In [286]:
#Output file file
file_name = output_folder+'covid.txt'

pheno_df[ pheno_df["famid"].notnull() ][cols_to_keep].drop_duplicates().to_csv\
(file_name,index=False, sep='\t')



In [310]:
write_famcount( pull_fam_count("covid", pheno_df),\
              "covid")

There are 1324 SOLAR families


In [288]:
pheno_df["pheno"] = "covid_noproband"

file_name = output_folder+'covid_noproband.txt'
pheno_df[ pheno_df["famid"].notnull() ][cols_to_keep].drop_duplicates().to_csv\
(file_name,index=False, sep='\t')



In [57]:
pheno_df["pheno"] = "covid_noproband"
write_famcount( pull_fam_count("covid_noproband", pheno_df, type_pheno="noproband"),\
              "covid_noproband")

pheno_df["pheno"] = "covid"

There are 5676 SOLAR families


## High Degree Families

In [29]:
high_deg_fams = pheno_df[ pheno_df["ptid"].astype(str).isin\
         (set( relationship_df[ relationship_df["degree"].isin\
                               (["Second","Third","Fourth"]) ]["mrn"]) )]["famid"].unique()

In [30]:
#Limit to higher degree
covid_notfirstdegree_df = pheno_df[ pheno_df["famid"].isin(high_deg_fams) ]

covid_notfirstdegree_df["pheno"] = "covid_highdeg_fam"

#Create no proband requirement
covid_notfirstdegree_nopro_df = covid_notfirstdegree_df.copy()
covid_notfirstdegree_nopro_df["pheno"] = "covid_highdeg_nopro"

In [59]:
write_famcount( pull_fam_count("covid_highdeg_fam", covid_notfirstdegree_df),\
              "covid_highdeg_fam")

There are 296 SOLAR families


In [58]:
write_famcount( pull_fam_count("covid_highdeg_nopro", covid_notfirstdegree_nopro_df, type_pheno="noproband"),\
               "covid_highdeg_nopro")

There are 962 SOLAR families


In [290]:
#Output file file
file_name = output_folder+'covid_high_degfam.txt'
covid_notfirstdegree_df[cols_to_keep].drop_duplicates().to_csv\
(file_name,index=False, sep='\t')




In [291]:
file_name = output_folder+'covid_high_degfam_noproband.txt'
covid_notfirstdegree_nopro_df[cols_to_keep].drop_duplicates().to_csv(file_name,index=False, sep='\t')


## Pull Time Data

In [32]:
#Get earliest PCR! 
pull_data_query = """ SELECT DISTINCT pat_mrn_id, MIN(order_time) AS order_time, MAX(order_time)
                      FROM user_kel2158.covid_lab_table
                      WHERE test_result = "pos" 
                    GROUP BY pat_mrn_id;  """

cur.execute(pull_data_query)
rawdata = cur.fetchall()

first_pcr_nohospreq_df = pd.DataFrame(list( rawdata ), columns=["mrn",
                           "order_time","latest_pcr_order_time"
                           ] )

#Get time difference between first and last positive test
first_pcr_nohospreq_df["pcr_timediff"] = (first_pcr_nohospreq_df["latest_pcr_order_time"] \
                         - first_pcr_nohospreq_df["order_time"] ).apply( lambda x: x.days ) 


len( first_pcr_nohospreq_df[ first_pcr_nohospreq_df["pcr_timediff"] > 30 ] )

#Percent patients with a second covid test after 30 days...
len( first_pcr_nohospreq_df[ first_pcr_nohospreq_df["pcr_timediff"] > 30 ] ) / len( first_pcr_nohospreq_df )

0.03731468531468531

In [33]:

#Every case should have an earlier positive amount
len( first_pcr_nohospreq_df ) == len(cases)

True

In [34]:
#Order by eariliest test
first_pcr_nohospreq_df.sort_values(by="order_time", inplace=True)

#A control is same at any time cut (neg test and no positive ever)
#Set pheno value 1 for cases (infected), 0 for controls (never infected)
first_pcr_nohospreq_df["value"] = 1

#Turn datetime into date
first_pcr_nohospreq_df["order_date"] = first_pcr_nohospreq_df["order_time"].apply(lambda x: x.date())

#Map famids to mrns
first_pcr_nohospreq_df["famid"] = first_pcr_nohospreq_df["mrn"].map( mrn2famid )

#Rename columns to align with pull family function
first_pcr_nohospreq_df.rename(columns={"mrn":"ptid"},inplace=True)
first_pcr_nohospreq_df.drop_duplicates(inplace=True)
#Remove those that aren't in fam file
first_pcr_nohospreq_df = first_pcr_nohospreq_df[ ~first_pcr_nohospreq_df["famid"].isnull() ]


## Pull Admissions Data

In [42]:
#Get first admission that is within 96 hours of test
#And 
pull_data_query = """

SELECT *
FROM (
    SELECT c.pat_mrn_id, a.encounter_start_datetime, a.encounter_censor_datetime,
    EXTRACT(HOUR FROM TIMEDIFF(l.order_time, a.encounter_start_datetime) ) AS hours_from_admission, 
    l.order_time, a.pat_enc_csn_id, c.event_type_desc

    FROM covid_ADT_defined_visits a,
	covid_ADT c,
        
        #Select first positive covid tests that's not an antigen test!
        ( SELECT DISTINCT pat_mrn_id, MIN(order_time) AS order_time
                      FROM user_kel2158.covid_lab_table
                      WHERE test_result = "pos" 
                    GROUP BY pat_mrn_id ) l

  WHERE c.pat_mrn_id = l.pat_mrn_id 
  AND c.pat_enc_csn_id = a.pat_enc_csn_id) d

WHERE
    #Covid test is within  96 hours of "admission", doesn't matter how long patient in hosp for, can be outpatient
    0 <= d.hours_from_admission 
    AND 96 >= d.hours_from_admission
;"""

cur.execute(pull_data_query)
rawdata = cur.fetchall()

first_admsn_df = pd.DataFrame(list( rawdata ), columns=["mrn",
                           "admsn_time","disch_time","hrs_from_admission"
                            ,"first_pos_result_datetime", "pat_enc","visit_type"
                           ] )


In [43]:
#Pull all admissions for patients with a positive pcr test
pull_data_query = """
               SELECT DISTINCT c.pat_mrn_id, a.encounter_start_datetime, a.encounter_censor_datetime, 
                l.order_time, a.pat_enc_csn_id, a.encounter_censor_type

                FROM covid_ADT_defined_visits a,
                covid_ADT c,

        #Select first positive covid tests that's not an antigen test!
        ( SELECT DISTINCT pat_mrn_id, MIN(order_time) AS order_time
                      FROM user_kel2158.covid_lab_table
                      WHERE test_result = "pos" 
                    GROUP BY pat_mrn_id ) l

              WHERE c.pat_mrn_id = l.pat_mrn_id
              AND c.pat_enc_csn_id = a.pat_enc_csn_id
             
;"""


cur.execute(pull_data_query)
rawdata = cur.fetchall()
admsn_df = pd.DataFrame(list( rawdata ),columns = ["mrn","hosp_admsn",
                                                 "hosp_disch","first_pos_result_datetime",
                                                 "pat_enc_csn_id","censor_stat"])

In [44]:
admsn_df["censor_stat"].unique()

array(['Discharge', 'Hospital Outpatient', 'Census',
       'Leave of Absence Census'], dtype=object)

In [45]:
#Closed admissions
discharged_admissions = admsn_df[ admsn_df["censor_stat"].isin\
                             (['Discharge', 'Hospital Outpatient']) ]

#Note admissions that are still open, will exclude for patients w/ days hospitalized potentially
open_admission = admsn_df[ admsn_df["censor_stat"].isin( \
                 ["Census", 'Leave of Absence Census' ])
                          & 
                         ~admsn_df["pat_enc_csn_id"].isin(discharged_admissions["pat_enc_csn_id"]) 
                         ]


In [46]:
len(open_admission)

161

## Pull DNI

In [321]:
# Individuals with DNI orders
pull_data_query ="""
SELECT pat_mrn_id, MIN(order_date) AS dnr_date
FROM covid_orders
WHERE description = 'DNR/DNI-DO NOT RESUSCITATE/DO NOT INTUBATE'
GROUP BY pat_mrn_id;
 """

cur.execute(pull_data_query)
rawdata = cur.fetchall()

mrn2dni_dnr = {e[0]:e[1] for e in rawdata}

In [322]:
#Pull intubation data
pull_data_query = """
SELECT DISTINCT pat_mrn_id,order_date
FROM covid_intubation_orders
WHERE order_date IS NOT NULL AND order_status != 'Canceled';"""
     
cur.execute(pull_data_query)

rawdata = cur.fetchall()

intub_df = pd.DataFrame(list(rawdata),columns=["mrn","intubation_date"] )

## Demo and Covariates
### Pull Data

In [323]:
#Pull addresses from covid

pull_data_query = """
 SELECT pat_mrn_id, add_line_1,zip
FROM covid_persons
"""

cur.execute(pull_data_query)
rawdata = cur.fetchall()

print ( len(rawdata) )
add_df_covid = pd.DataFrame(list(rawdata),
                columns=["mrn","address","zip"])


#Update to only include patients we have lab covid data for, since hhid file already created
#Note, no cumc_demogs data with address in 2020q4 db
pull_data_query = """
 SELECT mrn, add_line_1,zip
FROM clinical_merge_v5_240919.cumc_patient_demogs
WHERE mrn IN (
    SELECT DISTINCT l.pat_mrn_id
    FROM covid_labs l)
"""

cur.execute(pull_data_query)
rawdata = cur.fetchall()

print ( len(rawdata) )

add_df_2019 = pd.DataFrame(list(rawdata),
                columns=["mrn","address","zip"])


# First try using covid data (more up to date!)
add_df = pd.concat([ add_df_2019[ ~add_df_2019["mrn"].isin(add_df_covid["mrn"]) ], add_df_covid ])

add_df["address"].update( add_df["address"].str.upper() )

add_df.drop_duplicates(inplace=True)

print(len(add_df))

647231
438774
647231


In [324]:
#1 codes for "Female"
#Pull demo data from persons table if missing


pull_data_query = """SELECT p.pat_mrn_id, p.sex_c, p.race_1, p.ethnicity, YEAR(p.birth_date)
FROM covid_persons p,
    covid_labs l
WHERE p.pat_mrn_id = l.pat_mrn_id;"""

cur.execute(pull_data_query)
rawdata = cur.fetchall()


pull_data_query = """SELECT p2.pat_mrn_id, gender_source_value, race_source_value, ethnicity_source_value, year_of_birth
FROM person p,
    covid_patient2person p2,
    covid_labs l
WHERE 
    l.pat_mrn_id = p2.pat_mrn_id
    AND p2.person_id_2020 = p.person_id"""

cur.execute(pull_data_query)
rawdata1 = cur.fetchall()



#Dedup data
rawdata = set(rawdata)
rawdata1 = set(rawdata1)

#Add in covid demo data first
demo_covid_df = pd.DataFrame(list(rawdata), columns=["mrn","sex_raw","race1","ethnicity","birth_year"])

#Supplement with cumc demog data 

supplement_demo_df = pd.DataFrame(list(rawdata1), columns=["mrn","sex_raw","race1","ethnicity","birth_year"])

demo_df = pd.concat([supplement_demo_df[ ~supplement_demo_df['mrn'].\
                                        isin(demo_covid_df["mrn"].unique())] ,
                   demo_covid_df])

demo_df.drop_duplicates(inplace=True)

In [327]:
#Are we adding anything with supplementary data?
#False if we are adding anything
len( set( demo_covid_df['mrn'] ).union( set( supplement_demo_df['mrn'] ) ) ) == len( set( demo_covid_df['mrn'] ))

#Are we adding anything with supplementary data?
#Empty set if no
set( supplement_demo_df['mrn'] ) -  ( set( demo_covid_df['mrn'] ) )

set()

### Clean Address Data

In [328]:
#Shares house with someone who tests positive
address2mrnv2 = defaultdict(list)
mrn2address = dict()

add_inall_df =  add_df
add_inall_df.drop_duplicates(inplace=True)
add_inall_df.fillna("(null)",inplace=True)

for  row in tqdm(add_inall_df.iterrows()):
    #First row header
    if row[0] != 0:
        mrn = row[1]["mrn"]
        #Replace multiple spaces with one space
        address = re.sub( r'\s{2}' , " ", row[1]["address"] ) 
        house_num = address.split(" ")[0]
        #We want only the first part of zip since not all have full
        zipc = row[1]["zip"].split("-")[0]
        
        try:
            street =  address.replace(" W "," ")\
                                    .replace(" W. "," ")\
                                    .replace(" WEST "," ")\
                                    .replace(" E "," ")\
                                    .replace(" E. "," ")\
                                    .replace(" EAST "," ").split(" ")[1]
            hid = house_num+"_"+street+"_"+zipc
            
            #If missing a core component of hid or po box, don't add
            if ( (hid.find("null") == -1) & 
                (hid.find("__") == -1) &
                (hid.find("BOX_") == -1)& 
                (hid.find("PO_BOX") == -1) &
                (hid.find("POBOX") == -1) 
               ) :
                address2mrnv2[hid].append(mrn)
                mrn2address[mrn] = hid

        #No space to parse out street number
        #Updated to exclude, would rather have
        except:
            #hid = house_num+"_"+zipc
            continue

#         address2mrnv2[hid].append(mrn)
#         mrn2address[mrn] = hid


647231it [01:21, 7906.51it/s]


In [329]:
#List of buildings with more than one individual in it tested where 
#at least one person has covid
hid2covidcnt = dict()
mrn2covidcnt =  dict()

#For each building with mrns
for hid, mrns in tqdm( address2mrnv2.items() ):
    covid_count = 0
   
    #For each patient in building, check if had covid
    for mrn in mrns:
        if mrn in cases:
            covid_count += 1

    hid2covidcnt[hid] = covid_count
    
    #How many people in building other than self have covid?
    for mrn in mrns:
        if covid_count == 0:
            mrn2covidcnt[mrn] = covid_count
        
        #if patient has covid, remove one from count and assign 
        elif mrn in cases:
            mrn2covidcnt[mrn] = covid_count-1
        
        #if patient doesn't have covid assign total count
        else:
            mrn2covidcnt[mrn] = covid_count
        
    

100%|█████████████████████████████████████████████████████████████████████████████████| 283950/283950 [00:00<00:00, 286733.31it/s]


### Clean Demo Data

In [334]:
#Dictionary to map ethnicity to SOLARStrap friendly coding
map_ethnic_dict = { "NOT HISPANIC OR LATINO OR SPANISH ORIGIN":'U',
"DECLINED":'U',
"HISPANIC OR LATINO OR SPANISH ORIGIN":'H',
"AFRICAN AMERICAN":'B',
"MULTI-RACIAL":"Other",
"UNKNOWN":"U",
"CAUCASIAN":"W",
None:"U",
"(null)":'U',
"ASIAN / PACIFIC ISLANDER":"Other",
"AMERICAN INDIAN / ESKIMO":"Other"
                  }

map_race_dict = {'WHITE':'W'
                    , 'BLACK OR AFRICAN AMERICAN':'B'
                    , 'ASIAN':'Other'
                    ,'OTHER COMBINATIONS NOT DESCRIBED':'Other'
                   ,'AMERICAN INDIAN OR ALASKA NATION':'Other'
                   ,'NAT.HAWAIIAN/OTH.PACIFIC ISLAND':'Other'
                    ,'SEPHARDIC JEWISH':'Other'
                       ,'ASHKENAZI JEWISH':'Other'
                    ,'NULL':'NA'
                  , '(null)':'NA'
                  , 'DECLINED':'NA'
              }

#Available sex options
map_sex_dict = {2:'M',1:'F',3:'NULL',950:'NULL',
                "NULL":'NULL','(null)':'NULL',
                "M":"M","F":"F"}

In [335]:
#Clean race and sex
demo_df["race_code"] = demo_df["race1"].map(map_race_dict)
demo_df["race_code"].fillna(demo_df["race1"])
demo_df["sex"] = demo_df["sex_raw"].map(map_sex_dict)


pull_age = lambda x: np.nan if x is None else \
                 (np.nan if x == 0 else date.today().year - x)

demo_df["age"] = demo_df["birth_year"].apply(pull_age)

demo_df["ethnic_code"] = demo_df["ethnicity"].map(map_ethnic_dict)

#GET MISSING empis and add to demo with blank data
missing_demo_df = pd.DataFrame( pheno_df[ ~pheno_df["ptid"].\
                                         isin(demo_df["mrn"])]["ptid"].values, columns = ["mrn"])

missing_demo_df.drop_duplicates(inplace=True)
for col in demo_df.columns[1:]:
    missing_demo_df[col] = None
    
    
#Concat demo with missing demo dataframes
demo_all_df = pd.concat([missing_demo_df, demo_df])
demo_all_df.reset_index(inplace=True)

pull_decade = lambda x: None if x is None\
                 else (None if x =='0' else str(x)[:-1]+"0" )
demo_all_df["birth_decade"] = demo_all_df["birth_year"].apply(pull_decade)

#Fill with "NA" for race for SOLAR race/ethnicity Fe algorithm
demo_all_df["race_code"].fillna("NA",inplace=True)

#Fill with "NULL" for other demos if missing
fill_with_NULL = ["age","ethnic_code","sex"]

for demo in fill_with_NULL:
    demo_all_df[demo].fillna("NULL",inplace=True)




In [336]:
#Solar implementation on combining race and ethnicity
def fe_ethnicity_race(mrn, race, ethnicity):
    if race == 'NA':
        if ethnicity == 'W':
            race = 'White'
        elif ethnicity == 'B':
            race = 'Black'
        elif ethnicity in ('H', 'S', 'O'):
            race = 'Hispanic'
        elif ethnicity in ('NULL', '', 'U', 'D', '2'):
            race = 'Unknown'
        else:
            race = 'Other'
    else:
        if race == 'W':
            if ethnicity == 'H':
                race = 'Hispanic'
            else:
                race = 'White'
        elif race == 'B':
            if ethnicity == 'H':
                race = 'Hispanic'
            else:
                race = 'Black'
        elif race == 'O':
            race = 'Hispanic'
        elif race in ('U', 'D'):
            if ethnicity == 'H':
                race = 'Hispanic'
            else:
                race = 'Unknown'
        else:
            race = 'Other'
    mrn2race_fe[race][mrn] = 1

In [337]:
mrn2race_fe = defaultdict(dict)
#Apply fe's alg to rows in df

for row in demo_all_df.iterrows():
    fe_ethnicity_race(row[1]["mrn"], row[1]["race_code"],row[1]["ethnic_code"])


In [344]:
len(demo_all_df)

202798

In [343]:
#Output Demo file
file_name = filename_demo

demo_all_df[["mrn","sex","birth_decade","race_code","ethnic_code","age"]].\
to_csv(file_name,index=False, sep='\t', header=["ptid","sex","birth_decade","race_code",
                                           "ethnic_code","age"])





## Pheno: Cummulative

In [38]:
def output_df_function(df, filename_or_pheno, base_path):
#Output covid file
    file_name = base_path + filename_or_pheno + ".txt"
    
    if not os.path.isfile(file_name):
        df.to_csv(file_name,index=False, sep='\t',\
                  mode='a',columns=["ptid","pheno","value"] )
   
    else: # else it exists so append without writing the header
        df.to_csv(file_name,index=False, sep='\t',\
                   mode='a', header=False, columns=["ptid","pheno","value"] )



In [168]:
#Get list of pheno datecut dataframes to concat

def pull_datecut_dataframe(df, allow_class_switching=False):
    
    #Get earliest and latest test dates
    date = min( df["order_date"] )
    last_test_date = max( df["order_date"] )
    
    #All phenos 
    pheno_datecut_list = list()
    
    #Dates of phenos
    datecut_date_list = list()
    
    #Number of families with those phenos
    datecut_fam_num_list = list()
    
    naming_pheno = "covid_weekly_"

    while date < last_test_date:
        date += timedelta(days= 7)
        
        month = date.month
        year = date.year

        #get all first positive (for cases)
        case_df = df[ ( df["value"] == 1 ) &\
                        (df['order_date'] <= date)] 
        #get all controls irrespective of date cutoff - assume always a case if tested
        control_df = df_control.copy() 
        
        if allow_class_switching:
            naming_pheno = "covid_cswitch_weekly_"
            
            #These are patients that will be infected but jhavente been yet
            future_cases_df = df[ ( df["value"] == 1 ) &\
                (df['order_date'] > date)] 
            
            control_df = pd.concat([future_cases_df, control_df])        
            
            #Turn future cases into controls
            control_df["value"] = 0

        df_datecut = pd.concat([case_df, control_df])
        
        pheno = naming_pheno + date.strftime('%m_%d_%Y')
        df_datecut["pheno"] = pheno
        
        #Get the number of SOLAR families that qualify with that date cut (ie >=1 case and >1 case+control)
        fam_num = pull_fam_count(pheno, df_datecut, print_res = False)
        
        #Write the sample sizes
        write_famcount( fam_num, naming_pheno + date.strftime('%m_%d_%Y'))
        
        datecut_fam_num_list.append(fam_num)
        datecut_date_list.append( date )
        
        #Set minimum number of solar families to be 45
        if fam_num >= 45:
            pheno_datecut_list.append(df_datecut) 
            filename = naming_pheno +\
                        str( month ) +"_" + str( year)
            
            base_path = '/home/kel2158/Projects/COVIDHerit/data/12_29_21/cummulative/'
            
            if allow_class_switching:
                base_path = '/home/kel2158/Projects/COVIDHerit/data/12_29_21/cummulative_classswitch/'
            
            output_df_function(df_datecut, filename, base_path)
            
#     Concat 
    pheno_datecut_df = pd.concat(pheno_datecut_list)
    
    return( pheno_datecut_df )


In [126]:
#Create control dataframe
df_control = pd.DataFrame(controls,columns=["ptid"])
df_control["value"] = 0
df_control["order_time"] = np.nan
df_control["famid"] = df_control["ptid"].map(mrn2famid)

#Pull dataframe
pheno_datecut_df = pull_datecut_dataframe(first_pcr_nohospreq_df)

#Pull dataframe
pheno_classswitch_datecut_df = pull_datecut_dataframe(first_pcr_nohospreq_df, allow_class_switching=True)

In [172]:
print( f"Number of weeks with over 40 families when a control cannot switch to a case: {len( set( pheno_datecut_df['pheno']  ) )} " )
      
print( f"First week with over 40 families: { pheno_datecut_df['pheno'].values[0] } " )                                                         

Number of weeks with over 40 families when a control cannot switch to a case: 84 
First week with over 40 families: covid_weekly_03_24_2020 


In [170]:

print( f"Number of weeks with over 40 families when a control can switch to a case: {len( set( pheno_classswitch_datecut_df['pheno']  ) )} " )
      
print( f"First week with over 40 families: { pheno_classswitch_datecut_df['pheno'].values[0] } " )                                                         

Number of weeks with over 40 families when a control can switch to a case: 84 
First week with over 40 families: covid_cswitch_weekly_03_24_2020 


## Pheno: Days Hospitalized

In [354]:
def get_days(timedelta_date):

    days = timedelta_date.days
    partial_day = timedelta_date.seconds/(24*60*60)
    
    return(days+partial_day)

In [355]:
#We want to limit first hosp dataframe so every patient has just one initial start encounter for getting covid test
#Select on earliest admission time
#Why not just use defined_visits?  Some are wonky - ex pat_enc_csn_id = 147040327
grouped_firstadmin = first_admsn_df[["pat_enc","mrn","admsn_time","disch_time"]].drop_duplicates().groupby(["mrn"])

mrn2first_hospenc = dict()
mrn2first_hospenc_admsn = dict()
mrn2first_hospenc_disch = dict()

for group in tqdm( grouped_firstadmin):

    mrn = group[0]
    holder_df = group[1]
    
    #First filter by getting the earliest admission within 96 hours of first PCR test
    holder_df2 = holder_df[ holder_df["admsn_time"] == holder_df["admsn_time"].min() ] 

    #Then filter by getting the latest discharge time associated with that intake time
    #ie may transfer between centers etc
    holder_df3 = holder_df2[ holder_df2["disch_time"] == holder_df2["disch_time"].max() ] 

    #Sometimes identical patient encounters in terms of admission and discharge time (Different department location) 
    #Don't care about this so just take first encounter
    patient_encounter = holder_df3["pat_enc"].values[0]
    patient_admsn = holder_df3["admsn_time"].values[0]
    patient_disch = holder_df3["disch_time"].values[0]
    

    #Sometimes the admission time and discharge time is slightly different for a single encounter.
    mrn2first_hospenc[mrn] = patient_encounter
    mrn2first_hospenc_admsn[mrn] = patient_admsn
    mrn2first_hospenc_disch[mrn] = patient_disch

    
    #Making sure they are all in fact identical except for patient encounter number
    if len(holder_df3[["mrn","admsn_time","disch_time"]].drop_duplicates()) >1:
        print(holder_df3)
        break


#Limit first hosp dataframe so every patient has just one initial start encounter
first_admsn_limit_df = first_admsn_df[ ( first_admsn_df["mrn"].map(mrn2first_hospenc) == first_admsn_df["pat_enc"]) 
                                     & ( first_admsn_df["mrn"].map(mrn2first_hospenc_admsn) == first_admsn_df["admsn_time"])
                                     & ( first_admsn_df["mrn"].map(mrn2first_hospenc_disch) == first_admsn_df["disch_time"]) ]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11979/11979 [01:03<00:00, 187.92it/s]


In [356]:
#Drop duplicates
first_admsn_limit_df.drop(columns="visit_type",inplace=True,axis=1)
first_admsn_limit_df.drop_duplicates(inplace=True)

In [357]:
#Each mrn should have one first admission tied to a positive test
len(first_admsn_limit_df) == len(set(first_admsn_limit_df["mrn"]))

True

In [140]:
#Limit admissions to those that are discharged
admsn_df = discharged_admissions

In [359]:
#Identical patient encounter IDs with admission, transfer etc for visit type.  To drop visit_type and dedup

first_admsn_limit_df.drop_duplicates(inplace=True)


In [361]:
#Get time in hospital
admsn_df["hosp_days"] = admsn_df["hosp_disch"] - admsn_df["hosp_admsn"]
admsn_df["hosp_days"] = admsn_df["hosp_days"].apply(get_days)

#Sort values by admission time and then mrn
admsn_df.sort_values(["hosp_admsn"],ascending= True,inplace=True)

In [362]:
mrn2admsissions = defaultdict(dict)

for mrn in first_admsn_limit_df["mrn"]:
    #Only include those admissions that last longer than a day, otherwise don't care
    df = admsn_df[ (admsn_df["mrn"] == mrn) & (admsn_df["hosp_days"] >= 1) ]
    adm_lst = df["hosp_admsn"].to_list()
    disch_lst = df["hosp_disch"].to_list()
    
    mrn2admsissions[mrn]["admsn"] = adm_lst
    mrn2admsissions[mrn]["disch"] = disch_lst


In [365]:
#Loop through and see if re-admitted for covid
#ie if never admitted, admitted within 12 days
#if admitted, admitted within 5 days of previous discharge

mrn2final_disch = dict()
mrn2final_admission = dict()
mrn2hosp_days = dict()
mrn2already_hosp = dict()
mrn_openadmission = list()

for row in tqdm( first_admsn_limit_df.iterrows() ):
    mrn = row[1]["mrn"]
    hosp_days = float( row[1]["hrs_from_admission"] )
    initial_admsn_date = row[1]["admsn_time"]
    initial_disch_date = row[1]["disch_time"]
    
    
    #Assume only 1 admission/discharge related to covid start
    final_admsn = initial_admsn_date
    final_disch = initial_disch_date
    
    #Note, these are limited to admissions lasting more than 24 hours
    further_adm = mrn2admsissions[mrn]["admsn"]
    further_dis = mrn2admsissions[mrn]["disch"]
    
    #Reset hosp_days to 0,admitted an inconsequential amount of time (under a day) or outpatient
    if hosp_days < 1:
        hosp_days = 0
    
    num_admin = len(further_adm)
    
    
    #If patient has been in there less than a day
    #see if there is another admission within 12 days of initial discharge that lasted more than a day 
    if num_admin > 0:
        
        for admission in zip( further_adm,further_dis):
            nxt_admsn = admission[0]
            nxt_disch = admission[1]
            
            #Admission is before initial covid admission
            if nxt_admsn < initial_admsn_date:
                
                #Need to make sure patient wasn't already in hospital when they got the covid test
                if nxt_disch >= initial_admsn_date:
                    
                    #Problem, looks like patient was already in hospital prior to covid test and may have gotten it 
                    #There
                    mrn2already_hosp[mrn]  = [nxt_admsn,nxt_disch]
 
            #Re-admitted within 12 days of initial positive covid pcr test which had been outpatient/ER
            #ie initial hospital stay w/ covid was for less than a day
            elif hosp_days == 0:

                #Patient declines within 12 days of first positive and admitted (ie there at least a day)
                if get_days(nxt_admsn - final_disch) <= 12:

                    #Add additional days to total days in hospital
                    hosp_days += get_days(nxt_disch - nxt_admsn)
       

                    #Update final discharge date
                    final_disch = nxt_disch
                    final_admsn = nxt_admsn
                
                
            #There are sometimes overlapping admissions, see if current admission occurs before final discharge
            elif nxt_admsn < final_disch:
                
                #Check if subsequent discharge is later
                if nxt_disch > final_disch:
                    
                    #If so, need to add differences between discharges and update final discharge
                    hosp_days += get_days(nxt_disch - final_disch)
                    
                    #Assign this later discharge as final
                    final_disch = nxt_disch
                    
                    
            #Else patient was initially admitted, see if readmitted b/c of covid (ie within 5 days of previous)
            #This can be negative if overlapping admissions/discharges
            elif get_days(nxt_admsn - final_disch) <= 5:
                hosp_days += get_days(nxt_disch - nxt_admsn)
                final_disch = nxt_disch
                final_admsn = nxt_admsn
                
    mrn2final_disch[mrn] = final_disch
    mrn2final_admission[mrn] = final_admsn
    mrn2hosp_days[mrn] = hosp_days

            

11979it [00:02, 5278.53it/s]


In [374]:
first_admsn_limit_df["hosp_days"] = first_admsn_limit_df["mrn"].map(mrn2hosp_days)
first_admsn_limit_df["final_disch"] = first_admsn_limit_df["mrn"].map(mrn2final_disch)

#These are people that were hospitalized during time of covid test
first_admsn_limit_df["already_hosp"] = first_admsn_limit_df["mrn"].map(mrn2already_hosp)

#Map people that have died
mrn2death = dict(zip( deaths_df["mrn"],deaths_df["death_date"]))
first_admsn_limit_df["death_date"] = first_admsn_limit_df["mrn"].map(mrn2death)

In [378]:
#limit to patients that were not already in hospital prior to first positive test (ie over 96 hrs)
#And no open admissions
#And their final discharge is at least 12 days before data pull 
#And patient hasn't died within 5 days of discharge

data_pull = first_admsn_limit_df["final_disch"].max()

hosp_patient_df = first_admsn_limit_df[ ( first_admsn_limit_df["already_hosp"].isnull()) &
                    ( ~ first_admsn_limit_df["mrn"].isin( open_admission["mrn"] )) & 
                    ( (first_admsn_limit_df["final_disch"] + timedelta(12)) < data_pull ) &
                    ( ( first_admsn_limit_df["death_date"].isnull())  | \
                        ( first_admsn_limit_df["death_date"] - timedelta(5)  >  first_admsn_limit_df["final_disch"]))  
                                    
                                      ] 

In [ ]:
hosp_patient_df["famid"] = hosp_patient_df["mrn"].map(mrn2famid)

In [428]:
pheno_days_hosp = hosp_patient_df[[ "mrn","hosp_days","famid" ]]
pheno_days_hosp.rename(columns={"mrn":"ptid","hosp_days":"value"},inplace=True)
pheno_days_hosp["pheno"] = "days_hosp"

In [432]:
write_famcount( pull_fam_count("days_hosp", pheno_days_hosp),\
               "days_hosp")

#Output Outcomes
file_name = output_folder + "days_hosp.txt"

pheno_days_hosp[cols_to_keep].to_csv(file_name,index=False, sep='\t')



There are 115 SOLAR families


## Pheno: Intubation

In [381]:
#Combine admissions data with intubation data
#Dont care if patient has died or discharged or whatever
intub_hospcombined_df = intub_df.merge( first_admsn_limit_df,
                                  on="mrn")

#Make datetime (not time excluded)
intub_hospcombined_df["intubation_date"] = pd.to_datetime(intub_hospcombined_df["intubation_date"])

#Patients intubated after covid admission start (add a day since no time) with test, 
#but before discharge
intub_covidstay_df = intub_hospcombined_df[ 
    ( intub_hospcombined_df["intubation_date"] + timedelta(1) > intub_hospcombined_df["admsn_time"] ) &
( intub_hospcombined_df["intubation_date"] < intub_hospcombined_df["final_disch"] )  ]

In [382]:
#Cases are patients that have been intubated during a covid stay
mrn2intub_covidstay = dict(zip( intub_covidstay_df.mrn.unique(), 
                               len(intub_covidstay_df.mrn.unique())*[1]) )
#Controls are all patients with positive pcr test that don't have a dni/dnr

intub_pheno_df = pheno_df[pheno_df["value"] == 1][["ptid","famid"]]
#Being very conservative, irrespective of date/covid date, exclude if there is a dnr/dni order
intub_pheno_df["dni_dnr_date"] = intub_pheno_df["ptid"].map(mrn2dni_dnr)
intub_pheno_df["value"] = intub_pheno_df["ptid"].map(mrn2intub_covidstay)
intub_pheno_df["value"].fillna(0,inplace=True)
intub_pheno_df["pheno"] = "intubation"

#If hospitalized include final discharge
intub_pheno_df["final_disch_ifhosp"] =intub_pheno_df["ptid"].map(mrn2final_disch)

#Patients with DNI/DNR specified before discharge from covid stay
#and were intubated during covid stay
dnr_dni_duringcovidstay_df = intub_pheno_df[ ( intub_pheno_df["dni_dnr_date"].notnull() )& 
(intub_pheno_df["final_disch_ifhosp"] > pd.to_datetime( intub_pheno_df["dni_dnr_date"])) 
              ]
#Final dataframe, exclude patients with DNRs
intub_pheno_df = intub_pheno_df[~ (
    ( intub_pheno_df["ptid"].isin( dnr_dni_duringcovidstay_df["ptid"].unique() ) )
    & (intub_pheno_df["ptid"] == 0 ) 
                                    )]

intub_pheno_df.drop_duplicates(inplace=True)

In [457]:
write_famcount( pull_fam_count("intubation",intub_pheno_df),\
               "intubation")

There are 26 SOLAR families


## Pheno Death

In [383]:
#Cases are patients with positive covid test that die
#Controls are those that don;t die

death_pheno_df = pheno_df[pheno_df["value"] == 1][["ptid","famid"]]
#Being very conservative, irrespective of date/covid date, exclude if there is a dnr/dni order
death_pheno_df["dni_dnr_date"] = death_pheno_df["ptid"].map(mrn2dni_dnr)
death_pheno_df["death_datetime"] = death_pheno_df["ptid"].map(mrn2death)

#If hospitalized include final discharge
death_pheno_df["final_disch_ifhosp"] =death_pheno_df["ptid"].map(mrn2final_disch)
death_pheno_df["pheno"] = "death"


#Get patients who died within 30 days of covid hospitalization stay
death_covidhosp_df = death_pheno_df [ (death_pheno_df["death_datetime"].notnull()) & 
(death_pheno_df["death_datetime"]  < death_pheno_df["final_disch_ifhosp"]+ timedelta(30) ) ]

mrn2death_duringcovhosp = dict(zip( death_covidhosp_df["ptid"],len( death_covidhosp_df["ptid"])*[1] ))


#For now, don't care about DNI/DNR order
death_pheno_df["value"] = death_pheno_df["ptid"].map(mrn2death_duringcovhosp)
death_pheno_df["value"].fillna(0,inplace=True)


death_pheno_df.drop_duplicates(inplace=True)

In [458]:
write_famcount( pull_fam_count("death",death_pheno_df),\
               "death")

There are 0 SOLAR families


## Pheno: Hospitalization Status

In [384]:
#Hospital status where 0/1 for hospitalized or not
mrn2hosp_status = dict(zip( hosp_patient_df[ hosp_patient_df["hosp_days"] == 0]["mrn"] ,
         len(hosp_patient_df[ hosp_patient_df["hosp_days"] < 1]["mrn"])*[0]))

mrn2hosp_status.update( dict(zip( hosp_patient_df[ hosp_patient_df["hosp_days"] > 0]["mrn"] ,
         len(hosp_patient_df[ hosp_patient_df["hosp_days"] >= 1]["mrn"])*[1])))


hosp_patient_df["hosp_stat"] = hosp_patient_df["mrn"].map(mrn2hosp_status)

In [385]:
#Positive covid patients we have hosp info for
len(hosp_patient_df.mrn.unique())/len( pheno_df[ pheno_df["value"] == 1 ]["ptid"].unique())

0.6612027972027972

In [386]:
#Large falloff when we limit to those patients that get a covid test within
#96 hrs of admission (no limit in length of time)

#Additional falloff with patients excluded inculde (filtered out eaerlier in script): 
#1. have open admission, 
#2. are discharged within 12 days of data pull
#3. have died

len(first_admsn_df.mrn.unique())/len( pheno_df[ pheno_df["value"] == 1 ]["ptid"].unique())

0.6701538461538461

In [407]:
pheno_hospstat_df = hosp_patient_df[hosp_patient_df["hosp_stat"].notnull()][["mrn","hosp_stat","famid"]]

#Format
pheno_hospstat_df["pheno"] = "hosp_stat"
pheno_hospstat_df.rename(columns={"mrn": "ptid", "hosp_stat":"value" },inplace=True)


In [413]:
write_famcount( pull_fam_count("hosp_stat", pheno_hospstat_df),\
               "hosp_stat")

There are 115 SOLAR families


In [435]:
write_famcount( pull_fam_count("hosp_stat", pheno_hospstat_df),\
               "hosp_stat")

#Output hosp stat outcome
file_name = output_folder + "hosp_stat.txt"

#pheno_binary_df.to_csv(file,index=False, sep='\t')
pheno_hospstat_df[cols_to_keep].to_csv(file_name,index=False, sep='\t')



There are 115 SOLAR families


In [447]:
pheno_hospstat_df["pheno"] = "hosp_stat_nopro"

write_famcount( pull_fam_count("hosp_stat_nopro", pheno_hospstat_df,type_pheno="noproband" ),\
               "hosp_stat_nopro")


There are 158 SOLAR families


In [434]:
#Output with no proband
file_name = output_folder + "hosp_stat_noproband.txt"

pheno_hospstat_df[cols_to_keep].to_csv(file_name,index=False, sep='\t')

#Go back to normal
pheno_hospstat_df["pheno"] = "hosp_stat"
